In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor
from scipy.stats import randint
from sklearn.mixture import GaussianMixture
from sklearn import datasets, preprocessing
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings("ignore")

In [ ]:
DIR_INPUT = '/kaggle/input/tabular-playground-series-jul-2021'
trainset = pd.read_csv(DIR_INPUT + '/train.csv')
testset = pd.read_csv(DIR_INPUT + '/test.csv')
submission = pd.read_csv(DIR_INPUT +'/sample_submission.csv')

In [ ]:
features = testset.columns
targets = ["target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides"]

In [ ]:
  #trainset[['date','time']] = trainset.date_time.str.split(expand=True)
def categorize_time_of_day(dataframe):
    dataframe['date_and_time'] = pd.to_datetime(dataframe.date_time)  
    
    dataframe['day'] = dataframe.date_time.apply(lambda x: x.split(" ")[-1].split(":")[0]).astype(int)
#     dataframe.loc[(dataframe['day'] > 4) & (dataframe['day'] < 6), 'time_of_day'] = "dawn"
#     dataframe.loc[(dataframe['day'] >= 6) & (dataframe['day'] <= 10), 'time_of_day'] = "morning"
#     dataframe.loc[(dataframe['day'] > 10) & (dataframe['day'] <= 13), 'time_of_day'] = "noon"
#     dataframe.loc[(dataframe['day'] > 13) & (dataframe['day'] <= 15), 'time_of_day'] = "afternoon"
#     dataframe.loc[(dataframe['day'] > 15) & (dataframe['day'] <= 18), 'time_of_day'] = "evening"
#     dataframe.loc[(dataframe['day'] > 18) & (dataframe['day'] < 23), 'time_of_day'] = "night"
#     dataframe.loc[(dataframe['day'] > 22) | (dataframe['day'] <= 4), 'time_of_day'] = "midnight"
    dataframe.loc[(dataframe['day'] >= 6) & (dataframe['day'] <= 11), 'time_of_day'] = "morning"
    dataframe.loc[(dataframe['day'] > 11) & (dataframe['day'] <= 17), 'time_of_day'] = "afternoon"
    dataframe.loc[(dataframe['day'] > 17) & (dataframe['day'] <= 22), 'time_of_day'] = "evening"
    dataframe.loc[(dataframe['day'] > 22) | (dataframe['day'] <= 5), 'time_of_day'] = "night"

    dataframe = dataframe.drop(['day', 'date_time'], axis=1)
    dataframe['weekday'] = (dataframe.date_and_time.dt.dayofweek < 5).astype(int)
    
    return dataframe

trainset1 = categorize_time_of_day(trainset)
testset1 = categorize_time_of_day(testset)

In [ ]:
trainset1.head()

In [ ]:
for target in targets:
    fig, ax = plt.subplots(figsize=(20, 10))
    sns.scatterplot(data=trainset, x="deg_C", y=target, hue="time_of_day")
    plt.show()

In [ ]:
X=trainset1[["deg_C","target_benzene"]]
sc=preprocessing.StandardScaler()
sc.fit(X)
X_norm=sc.transform(X)
# クラスタリング
cls = KMeans(n_clusters=2)
result = cls.fit(X_norm)
# 結果を出力
plt.scatter(X_norm[:,0],X_norm[:,1], c=result.labels_)
plt.scatter(result.cluster_centers_[:,0],result.cluster_centers_[:,1],s=250, marker='*',c='red')
plt.show()